# QuantLib 簡介

QuantLib 提供了許多金融數學計算的功能，而這篇文章主要會涵蓋基礎的類別及功能

In [1]:
import QuantLib as ql

## 時間子模組

QuantLib 提供了一個名為 ql/time 的子資料夾，裡面實作了各種時間相關的類別。我們可以看到 Date 物件可以透過給定日期、月份、年份來建立：

In [2]:
# 創建一個日期，日期為 2015 年 3 月 31 日
date = ql.Date(31, 3, 2015) 

# 取出日期中的日、月、年，以及判斷該日期為星期幾
print(date.dayOfMonth())  # 輸出：31
print(date.month())  # 輸出：3
print(date.year())  # 輸出：2015
print(date.weekday() == ql.Tuesday)  # 輸出：True

# 日期的運算
print(date + 1)  # 在日期中加入一天，輸出：2015 年 4 月 1 日
print(date - 1)  # 在日期中減去一天，輸出：2015 年 3 月 30 日
print(date + ql.Period(1, ql.Months))  # 在日期中加入一個月，輸出：2015 年 4 月 30 日
print(date + ql.Period(1, ql.Weeks))  # 在日期中加入一週，輸出：2015 年 4 月 7 日
print(date + ql.Period(1, ql.Years))  # 在日期中加入一年，輸出：2016 年 3 月 31 日

# 日期的比較
print(ql.Date(31, 3, 2015) > ql.Date(1, 3, 2015))  # 輸出：True


31
3
2015
True
April 1st, 2015
March 30th, 2015
April 30th, 2015
April 7th, 2015
March 31st, 2016
True


我們可以使用 Schedule 物件來建立日期的序列，例如償還債券的日期。以下是一個例子：
這邊我們創建了一個 Schedule 物件，包含了從 date1 到 date2 的日期，並且使用 tenor 指定每月的間隔。calendar 則是用來確定哪些日期為假日。接著的兩個參數是業務日期轉換的慣例，這邊我們選擇的是遇到假日就選擇下一個非假日，所以你可以看到在日期序列中沒有假日的日期。

In [4]:
date1 = ql.Date(1, 1, 2015)
date2 = ql.Date(1, 1, 2016)
tenor = ql.Period(ql.Monthly)  # 指定每月的期間
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)  # 指定美國的曆法

# 創建一個行程表物件
schedule = ql.Schedule(date1, date2, tenor, calendar, ql.Following, ql.Following, ql.DateGeneration.Forward, False)

# 印出行程表中的日期
print(list(schedule))

[Date(2,1,2015), Date(2,2,2015), Date(2,3,2015), Date(1,4,2015), Date(1,5,2015), Date(1,6,2015), Date(1,7,2015), Date(3,8,2015), Date(1,9,2015), Date(1,10,2015), Date(2,11,2015), Date(1,12,2015), Date(4,1,2016)]


## 利率

InterestRate 類別可以用來儲存利率，並且指定複利類型、日計數慣例以及複利頻率。以下是創建一個年化利率為 5.0%，該利率按實際天數/實際天數的ISDA約定進行計算並進行年複利。

In [6]:
annualRate = 0.05
dayCount = ql.ActualActual(ql.ActualActual.ISDA)
compoundType = ql.Compounded
frequency = ql.Annual
interestRate = ql.InterestRate(annualRate, dayCount, compoundType, frequency)

在利率為 interestRate 的條件下，若你投資了一美元，則 compoundFactor 函數可以算出在 t 年後你的投資價值為多少。下方是驗證 compoundFactor 計算結果與預期的複利公式是否一致：

In [7]:
print(interestRate.compoundFactor(2.0))  # 輸出：1.1025
print((1.0 + annualRate)*(1.0 + annualRate))  # 檢查上述計算，輸出：1.1025

1.1025
1.1025


discountFactor 函數則是回傳 compoundFactor 函數的倒數。折現因子在計算未來現金流的現值時非常有用：

In [8]:
print(interestRate.discountFactor(2.0))  # 輸出：0.9070294784580498
print(1.0 / interestRate.compoundFactor(2.0))  # 輸出：0.9070294784580498

0.9070294784580498
0.9070294784580498


我們也可以將給定的利率轉換成其他類型的利率：

In [9]:
newFrequency = ql.Semiannual
effectiveRate = interestRate.equivalentRate(compoundType, newFrequency, 1)
print(effectiveRate.rate())

0.04939015319191986


這邊我們將利率轉換成半年複利的型式。4.939% 的半年複利等於 5.0% 的年複利。也就是說，兩者的折現因子應該要相同，我們來驗證一下：這證明了不管我們使用哪種利率轉換方式，計算出的淨現值應該都是一致的（忽略某些精度問題）。

In [10]:
print(interestRate.discountFactor(1.0))  
print(effectiveRate.discountFactor(1.0)) 

0.9523809523809523
0.9523809523809521


## 結論

這篇文章我們瞭解了如何使用 QuantLib 的基礎功能：

我們學會了如何使用時間子模組中的 Date 和 Schedule 類別。
我們學會了如何使用 InterestRate 類別。
這些基礎的功能為金融數學的計算提供了強大的工具，讓我們可以專注在金融模型的建立上，而不需要過多地擔心基礎的計算問題。